# Sample calculation
[https://github.com/rdmorgnzation/soilbearing/blob/main/notebooks/sample.ipynb](https://github.com/rdmorgnzation/soilbearing/blob/main/notebooks/sample.ipynb)

In [1]:
#Other imports from library
import resources #Must be 1st line
from blcalc import root_dir
from blcalc.excel_load import BoreholeDataSheets
from blcalc.borehole_parser import BoreholeLog
from blcalc.material import LayerSoil, MaterialData
from blcalc.footing import Footing, FootingType, FootingData
from blcalc.assembly import Assembly
from blcalc.solver import Solver, Methods
from blcalc.geocode import GeoCoder
import pandas as pd
from blcalc.soilproperty import SoilProperty

In [2]:
filepath = resources.test_data_dir() / "bh1.xls"

#### Borehole log:
[https://github.com/rdmorgnzation/soilbearing/blob/main/media/media/testdata/bh1.xls](https://github.com/rdmorgnzation/soilbearing/blob/main/media/media/testdata/bh1.xls)
<img src="./excel_screenshot/1.png">
<img src="./excel_screenshot/2.png">
<img src="./excel_screenshot/3.png">
#### Summary sheet:
[https://github.com/rdmorgnzation/soilbearing/blob/main/media/media/testdata/ss.xls](https://github.com/rdmorgnzation/soilbearing/blob/main/media/media/testdata/ss.xls)
<img src="./excel_screenshot/ss.png">

In [3]:
#Parse it first
sheets = BoreholeDataSheets.load_file(str(filepath.resolve()))

In [4]:
#Parse 1st sheet
keys = list(sheets.keys())
sheet = sheets[keys[0]]
borehole_log = BoreholeLog(sheet)

In [5]:
#Attributes obtained from data
pd.DataFrame.from_dict(borehole_log.attributes, orient='index')

,0
project,Soil Investigation of Proposed Rastriya Banijy...
location,"Thapathali, Kathmandu"
consultants,<Removed>
bore hole no,BH-1
"diameter of bh, mm",100 mm
rl of gwt,1.50 m
date,<Removed>
logged by,<Removed>
checked by,<Removed>
certified by,<Removed>


In [6]:
#Parsed values
layer=borehole_log.values
#Here i am removing the last value since we used values <20 only,
# For liquefaction. I am trying to get same result as in report
layer = list(filter(lambda x:x[SoilProperty.depth]<20.,layer))
layer = list(filter(lambda x:x[SoilProperty.depth]%1.5==0,layer))
#just for consistancy
pd.DataFrame.from_records(layer)

,SoilProperty.depth,SoilProperty.SPT_N,SoilProperty.GI
0,1.5,4.0,FM
1,3.0,6.0,FM
2,4.5,7.0,MI
3,6.0,9.0,MI
4,7.5,9.0,MI
5,9.0,10.0,MI
6,10.5,10.0,MI
7,12.0,10.0,MI
8,13.5,10.0,MI
9,15.0,10.0,MI


#### Geocoding

In [7]:
geocoder = GeoCoder()
result= geocoder.fetch_geo('Rastriya Banijya Bank, Thapathali, Kathmandu')
#Or use google maps to get result
if result:
    print('Latitude: ', result[0], '(27.655862)')
    print('Longitude: ', result[1], '(85.3064456)')

Latitude:  27.6965195 (27.655862)
Longitude:  85.30662 (85.3064456)


#### Add datas from summary sheet and create footing info

In [8]:
#Create footing info based on depth
def get_footing(depth=1.5):
    footing = Footing()
    footing[FootingData.Type] = FootingType.Square
    footing[FootingData.Depth] = depth
    footing[FootingData.Width] = 2
    footing[FootingData.Length] = 2
    return footing
#In our case these values were loaded from excel directly
water_depth = 1.5
for i in layer:
    i[SoilProperty.gamma] = 1.824
    i[SoilProperty.phi] = 32
    i[SoilProperty.qu] = 95
    i[SoilProperty.cu] = 4.5
    i[SoilProperty.G] = 2.511
    if i[SoilProperty.depth]<=3:
        i[SoilProperty.FC] = 53
        i[SoilProperty.water_per] = 0.1749
    else:
        i[SoilProperty.FC] = 95
        i[SoilProperty.water_per] = 0.3376
pd.DataFrame.from_records(layer)

,SoilProperty.depth,SoilProperty.SPT_N,SoilProperty.GI,SoilProperty.gamma,SoilProperty.phi,SoilProperty.qu,SoilProperty.cu,SoilProperty.G,SoilProperty.FC,SoilProperty.water_per
0,1.5,4.0,FM,1.824,32,95,4.5,2.511,53,0.1749
1,3.0,6.0,FM,1.824,32,95,4.5,2.511,53,0.1749
2,4.5,7.0,MI,1.824,32,95,4.5,2.511,95,0.3376
3,6.0,9.0,MI,1.824,32,95,4.5,2.511,95,0.3376
4,7.5,9.0,MI,1.824,32,95,4.5,2.511,95,0.3376
5,9.0,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376
6,10.5,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376
7,12.0,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376
8,13.5,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376
9,15.0,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376


#### Now compute values for each layer

In [9]:
soil_material = LayerSoil(layer, water_depth, 0)
#Here 0 is for internal purpose (PLAXIS result caching)
pd.DataFrame.from_records(soil_material.get())

,SoilProperty.depth,SoilProperty.SPT_N,SoilProperty.GI,SoilProperty.gamma,SoilProperty.phi,SoilProperty.qu,SoilProperty.cu,SoilProperty.G,SoilProperty.FC,SoilProperty.water_per,SoilProperty.sat_unit_weight,SoilProperty.vertical_effective_stress,SoilProperty.total_effective_stress,SoilProperty.water_depth,SoilProperty.thickness,SoilProperty.N60,SoilProperty.packing_case,SoilProperty.elasticity,SoilProperty.nu
0,1.5,4.0,MF,1.824,32,95,4.5,2.511,53,0.1749,20.109596,26.840160,26.840160,1.5,1.5,4.363333,1,4363.333333,0.3
1,3.0,6.0,MF,1.824,32,95,4.5,2.511,53,0.1749,20.109596,42.289554,57.004554,1.5,1.5,6.203635,1,6203.635233,0.3
2,4.5,7.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,54.323002,83.753002,1.5,1.5,7.237280,1,7237.280059,0.3
3,6.0,9.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,66.356450,110.501450,1.5,1.5,9.409680,1,9409.679640,0.3
4,7.5,9.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,78.389897,137.249897,1.5,1.5,8.657378,1,8657.377609,0.3
5,9.0,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,90.423345,163.998345,1.5,1.5,8.956403,1,8956.402748,0.3
6,10.5,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,102.456793,190.746793,1.5,1.5,8.856863,1,8856.862762,0.3
7,12.0,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,114.490241,217.495241,1.5,1.5,8.378496,1,8378.495871,0.3
8,13.5,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,126.523688,244.243688,1.5,1.5,7.970111,1,7970.111042,0.3
9,15.0,10.0,MI,1.824,32,95,4.5,2.511,95,0.3376,17.832298,138.557136,270.992136,1.5,1.5,7.616156,1,7616.156245,0.3


#### Sample calculation for 1.5m depth
Saturated Unit Weight ($\gamma_{sat}$)
= $9.81*\frac{(1+w\%)* G }{ 1+ w\% * G}$
= $9.81*\frac{(1+0.1749)*2.511}{1+0.1749*2.511}$
= 20.10959618
<br/>
Vertical Effective Stress=$
\begin{cases}
q + (\gamma_{sat} - 9.81)*depth & ,for below WL \\
q + \gamma * 9.81 * depth & ,for above WL
\end{cases}
$<br/>
So, Vertical Effective Stress = 0 + 1.824 * 9.81 * 1.5 = 26.84016

Total Effective Stress=$
\begin{cases}
q + \gamma_{sat} *depth & ,for below WL \\
q + \gamma * 9.81 * depth & ,for above WL
\end{cases}
$<br/>
So, Total Effective Stress = 0 + 1.824 * 9.81 * 1.5 = 26.84016

##### Determining N60
SPT_N = 4.0

Apply dilatracy correction,<br/>
Since SPT_N < 15 and above GWT no correction is needed,<br/>
else, $N = 15 + 0.5 * (SPT\_N - 15)$<br/>
So, N = 4.0

And,
Cb = 1<br/>
Cs = 1<br/>
Eh = 0.55<br/>
Cr = 0.7 for depth < 3 <br/>
And, CN = $9.78*\sqrt{\frac{1}{Vertical Effective Stress}} \le 1.7 $<br/> = $9.78 * \sqrt{\frac{1}{26.084016}}$ = 1.91492 > 1.7 = 1.7<br/>
So, $N_{60} = Cb*Cs*Eh*Cr*CN*N/0.6 = 1*1*0.55*0.7*1.7*4.0/0.6 = 4.3633333$

##### Packing Case
For sand N60, 4-10, packing case = 2<br/>
Starting count from 0(computer) = 2-1=1

##### Nu
$\nu$ = 0.3, for cohesionless soil(assumed)
##### Elasticity
$E = 10*N_{60}*100$ = 10 * 4.3633333*100 = 4363.3333

### Now calculate BC and LPI using diffrent methods
For 1.5m, avg_N60 = weighted average N60 from d/2 to 2d

In [10]:
soil_material.get_avg_N(1.5)

4.363333333333333

In [11]:
footing = get_footing()
assembly = Assembly(footing, soil_material)
solver = Solver(assembly)
results = solver.run()
#print(results)

## Shear Methods
### Terzaghi
$Rw1 = 0.5* (1+water\_depth/depth\_footing) = 1$<br/>
$Rw2 = 0.5* (1+(water\_depth-depth\_footing)/width\_footing) = 0.5$<br/>
$q_s = 1.2*c*N_c + surchage*N_q*Rw1 + 0.4*\gamma*9.81*width\_footing*N_\gamma*Rw2$<br/>
$=1.3*4.5*44.0357+26.840160*28.51657*1+0.4*1.824*9.81*2*28.78*0.5$<br/>
$=1228.987$<br/>
$q_{ns} = (q_s - q)/3 = (1228.987-26.84016)/3=400.7157$<br>

In [12]:
results[Methods.Terzaghi]

400.71581138243215

### Meyerhof
$K_p = tan^2(45+ \phi/2)$ = $tan^2(45+32/2)$ = 3.2546<br/>
sc = $1 + 0.2*K_p*width\_footing/length\_footing$ = $1+0.2*3.2546*2/2$ = 1.65092<br/>
$sq=1+0.1*K_p*width\_footing/length\_footing = 1+0.1*3.2546*2/2 = 1.32546$<br/>
$sy = sq = 1.32546$<br/>
$dc = 1 +0.2*\sqrt{K_p}*depth\_footing/width\_footing = 1+0.2*\sqrt{3.2546}*1.5/2 = 1.2706$<br/>
$dq =1 +0.1*\sqrt{K_p}*depth\_footing/width\_footing = 1+0.1*\sqrt{3.2546}*1.5/2 = 1.1353$<br/>
$dy = dq = 1.1353$<br/>
$c\_term = cohesion*Nc(phi)*sc*dc = 4.5*35.49026*1.65092*1.2706 = 335.009$<br/>
$q\_term = surchage*Nq(phi)*sq*dq*Rw1 = 26.84016*23.176776*1.32546*1.1353*1 = 936.0852$<br/>
$y\_term = 0.5*gamma*9.81*self.width\_footing*Ny(phi)*sy*dy*Rw2 = 0.5*1.824*9.81*2*22.02249*1.32546*1.1353*0.5 = 296.488$<br/>
$q_s=1567.58248$<br/>
$q_{ns}=(1567.58248-26.84016)/3=513.58$

In [13]:
results[Methods.Meyerhof]

513.5822200901385

### Hansen
$N_c=35.490260$<br/>
$N_q=23.1767762$<br/>
$N_y=20.78638$<br/>
$sc=1+N_q/N_c*width\_footing/length\_footing= 1+23.1767762/35.49026*2/2=1.653046$<br/>
$sq=1+width\_footing/length\_footing*tan(\phi)=1+2/2*tan(32)=1.624869$<br/>
$sy=1-0.4*width\_footing/length\_footing=1-0.4*2/2=0.6$<br/>
$dc = 1 +0.4*width\_footing/length\_footing=1+0.4*2/2=1.4$<br/>
$dq = 1 +2*tan(\phi)*(1-sin(\phi))^2 * depth\_footing/width\_footing=1+2*tan(32)*(1-sin(32))^2*1.5/2=1.20712$<br/>
$dy=1$<br/>
$c\_term = cohesion*N_c*sc*dc=4.5*35.49026*1.653046*1.4=369.6023$<br/>
$q\_term = surchage*N_q*sq*dq*Rw1=26.84016*23.1767762*1.624869*1.20712*1=1220.1323$<br/>
$y\_term = 0.5*gamma*9.81*width\_footing*N_y*sy*dy*Rw2=0.5*1.824*9.81*2*20.78638*0.6*1*0.5=111.581953$<br/>
$q_s = c\_term+q\_term+y\_term=1701.31656$<br/>
$q_{ns} = (1701.31656 - 26.84016)/3 = 558.158$

In [14]:
results[Methods.Hansen]

558.1594404871763

### Vesic
$N_c=35.4902607$<br/>
$N_q=23.176776$<br/>
$N_y=30.2146529$<br/>
$sc=1+N_q/N_c*width\_footing/length\_footing= 1+23.176776/35.4902607*2/2=1.653046$<br/>
$sq=1+width\_footing/length\_footing*tan(\phi)=1+2/2*tan(32)=1.624869$<br/>
$sy=1-0.4*width\_footing/length\_footing=1-0.4*2/2=0.6$<br/>
$dc = 1 +0.4*width\_footing/length\_footing=1+0.4*2/2=1.4$<br/>
$dq = 1 +2*tan(\phi)*(1-sin(\phi))^2 * depth\_footing/width\_footing=1+2*tan(32)*(1-sin(32))^2*1.5/2=1.20712$<br/>
$dy=1$<br/>
$c\_term = cohesion*N_c*sc*dc=4.5*35.49026*1.653046*1.4=369.6023$<br/>
$q\_term = surchage*N_q*sq*dq*Rw1=26.84016*23.1767762*1.624869*1.20712*1=1220.1323$<br/>
$y\_term = 0.5*gamma*9.81*width\_footing*N_y*sy*dy*Rw2=0.5*1.824*9.81*2*30.2146529*0.6*1*0.5=162.19322$<br/>
$q_s = c\_term+q\_term+y\_term=1751.93$<br/>
$q_{ns} = (1701.31656 - 26.84016)/3 = 575.03$

In [15]:
results[Methods.Vesic]

575.0298611603204

### Plaxis
For shear failure in PLAXIS, a large deformation is applied (same result can be obtained by applying large load), so that soil fails by shear. In both cases the soil fails during stagged loading. So we obtain result in plaxis which shows that step failed, but we get result.

In [16]:
results[Methods.PlaxisShear]

797.05

### Teng
$q_{ns} = 0.11*N60*N60*width\_footing*Rw2+0.33*(100+N60*N60)*depth\_footing*Rw1$<br/>
$= 0.11*4.36333^2*2*0.5+0.33*(100+4.36333^2)*1.5*1 = 61.018$

In [17]:
results[Methods.Teng]

61.01840005555556

## Settlement Methods
Take 25mm deflection,

### Meyerhof
$Rd = 1 + 0.33*depth\_footing/width\_footing = 1+0.33*1.5/2= 1.2475$<br/>
$b = water\_depth-depth\_footing = 1.5-1.5=0$<br/>
$Wy = 0.5+0.5*b / width\_footing=0.5$<br/>
$q_s = 8.1*n60*((width\_footing+0.3)/width\_footing)^2*Wy*Rd$<br/>
$=8.1*4.3633*((2+0.3)/2)^2*0.5*1.2475=29.154772$

In [18]:
results[Methods.MeyerhofDeflection]

29.154772040624994

### Bowels
$q_s = 1.5*Meyerhoff = 1.5*29.154772= 43.732158$

In [19]:
results[Methods.Bowels]

43.73215806093749

### Peck
$Cw = 0.5 + 0.5*water\_depth/(depth\_footing+width\_footing) = 0.5+0.5*1.5/(1.5+2) = 0.71429$<br/>
$q_s =  10.25*Cw*n60 = 10.25*0.71429*4.3633 = 31.94578$

In [20]:
results[Methods.Peck]

31.945833333333333

### Teng
$Rd = 1 + depth\_footing/width\_footing = 1 + 1.5/2 = 1.75$<br/>
$q_s =  35*(n60-3)*((width\_footing+0.3)/(2*width\_footing))^2*Wy*Rd$<br/>
$ = 35*(4.3633-3)*((2+0.3)/(2*2))^2*0.5*1.75= 13.803945$

In [21]:
results[Methods.TengDeflection]

13.804282552083333

### Plaxis
25mm deflection was applied at the footing depth and the corresponding force was taken as bearing capacity.

In [22]:
results[Methods.Plaxis]

40.06205

### LPI
Excel sheet: [https://github.com/rdmorgnzation/soilbearing/blob/main/media/media/testdata/lqi.xlsx](https://github.com/rdmorgnzation/soilbearing/blob/main/media/media/testdata/lqi.xlsx)
<br/>
For 1.5m depth,
$FC=94.36$<br/>
$delN1 = e^(1.63+(9.7/(FC+0.01))-(15.7/(FC+0.01))*(15.7/(FC+0.01)))=5.501985976$<br/>
$N1cs = N60+delN1 = 20.23473097$<br/>
$MSF = 0.8758$<br/>
$Csigma = 1/(18.9-2.55*\sqrt{N1cs}) = 0.134601693$<br/>
$Ksigma = 1-Csigma*log(vertical\_effective\_stress/100)= 0.921469899$<br/>
$CRR = exp((N1cs/14.1)+(N1cs/126)^2-(N1cs/23.6)^3+(N1cs/25.4)^3-2.8/1)*(MSF*Ksigma)=0.186699144$<br/>
$a_g = 0.183$<br/>
$SRF = exp(-1.012-1.126*sin(depth/11.73+5.133)+8*(0.106+0.118*sin(depth/11.28+5.142)))=0.998325105$<br/>
$CSR = 0.65*(total\_effective\_stress/vertical\_effective\_stress)*a_g*SRF = 0.118750771$<br/>
$FS = CRR / CSR = 0.992381257$<br/>
Since, 0.95<FS<1.2:<br/>
$Fz = 2e6*exp(-18.427*FS) = 0.022869488$<br/>
$Wz = 10 - data[SoilProperty.depth]/2 = 9.25$<br/>
$del\_lpi = Fz * Wz * thickness = 0.317314139$<br/>
Adding upto depth of 20 we get, LPI = 21.17671875

In [23]:
results[Methods.Liquifaction]

21.17671874695882